# Discussion 01

## Setup

First, download the required dependencies (at their respective versions). After running the below cell, you'll need to **restart your kernel** by clicking `"Kernel" > "Restart Kernel"` at the top bar.

In [ ]:
%pip install pandas==2.0.3
%pip install sqlalchemy==2.0.32
%pip install ipython-sql

Running the below cells creates the sqlite3 database `sqlite3:///books.db`.

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import make_books

## Discussion Questions

Database schema:

```
Books (book id, book name, year, genre, author id, times trending)
Authors (author id, author name, publishing company, debut year)
Awards (award id, book id, author name)
```

Run the below cells to start a sql connection.

In [12]:
%load_ext sql

In [13]:
%sql sqlite:///books.db

## 1) SQL Review
Suppose a full database schema is as follows:
- books (book id, book name, year, genre, author id, times trending)
- authors (author id, author name, publishing company, debut year)
- awards (award id, book id, award name, author name)

Below is how you will be writing your SQL queries with ["SQL magic"](https://www.datacamp.com/tutorial/sql-interface-within-jupyterlab).

In [14]:
%%sql
-- your code here --
;

 * sqlite:///books.db
0 rows affected.


[]

Write a SQL query to accomplish each task below:


**1. Select only the book names and years from the books table.**

In [15]:
%%sql
SELECT book_name, year FROM books;

 * sqlite:///books.db
Done.


book_name,year
And Then There Were None,1939
Endless Night,1967
Teaching to Transgress,1994
Chanwoo's Photobook,2017
The Dispossessed,1974
The Ministry for the Future,2020


**2. Find the names of all authors whose debut years are in the 21st century (defined as after 2000).**

In [16]:
%%sql
SELECT author_name
FROM authors
WHERE debut_year >= 2001;

 * sqlite:///books.db
Done.


author_name
Chanwoo


**3. Find the names of all authors who released a sci-fi genre book in 1974 that won an award.**

In [17]:
%%sql
SELECT aw.author_name
FROM books AS b
    INNER JOIN awards AS aw
    ON b.book_id = aw.book_id
WHERE b.genre = "sci-fi" AND b.year = 1974;

 * sqlite:///books.db
Done.


author_name
Ursula K. Le Guin
Ursula K. Le Guin


## 2) SQL Aggregation Review
In some cases, we may need to accumulate results by combining multiple rows via aggregation. We use the same schema as in Section 2.

**4. Write a query that calculates the total number of books written by author Agatha Christie.**

In [18]:
%%sql
SELECT COUNT(*)
FROM books AS b
    INNER JOIN authors AS a
    ON b.author_id = a.author_id
WHERE a.author_name = "Agatha Christie";%%sql

 * sqlite:///books.db
Done.
(sqlite3.OperationalError) near "%": syntax error
[SQL: %%sql]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


**5. The group-by construct lets us compute aggregates for different chunks of a relation. Find the total number of books released per genre. Don’t include genres with a count less than 2. (Hint: HAVING)**

In [19]:
%%sql
SELECT genre, COUNT(*)
FROM books
GROUP BY genre
HAVING COUNT(*) >= 2;

 * sqlite:///books.db
Done.


genre,COUNT(*)
mystery,2
nonfiction,2
sci-fi,2


## 3) Optional Questions

**6. Find the names of the 5 books that trended the least, ordered from least to most. Break ties by book name in alphabetical order.**

In [20]:
%%sql
SELECT book_name
FROM books
ORDER BY times_trending ASC, book_name ASC
LIMIT 5;

 * sqlite:///books.db
Done.


book_name
Chanwoo's Photobook
And Then There Were None
Teaching to Transgress
Endless Night
The Ministry for the Future


**7. *Challenge:* Write a query that computes the total times each author has had a trending book. (Hint: you will need to use a join for this as well.)**

In [21]:
%%sql
SELECT a.author_name, SUM(b.times_trending)
FROM books AS b
    INNER JOIN authors AS a
    ON b.author_id = a.author_id
GROUP BY a.author_name;

 * sqlite:///books.db
Done.


author_name,SUM(b.times_trending)
Agatha Christie,25
Chanwoo,0
Kim Stanley Robinson,15
Ursula K. Le Guin,20
bell hooks,10
